In [ ]:
print("hi")

In [ ]:
# Load all queries and correct answers from MATH-500 dataset into a dictionary
# The info is in the file normalized_datasets.json

import json
from pathlib import Path

# Load the normalized datasets file
dataset_path = Path("/home/adisi/repos/py/inter_2025/normalized_datasets.json")
with open(dataset_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract MATH-500 data - each entry has 'question', 'answer', and 'originalattr'
math_500_entries = data.get("MATH-500", [])

# Create a dictionary mapping questions to their correct answers
# Using question as key and answer as value
math_500_dict = {entry["question"]: entry["answer"] for entry in math_500_entries}

print(f"Loaded {len(math_500_dict)} questions and answers from MATH-500 dataset")
print(f"Example question: {list(math_500_dict.keys())[0][:100]}...")
print(f"Example answer: {list(math_500_dict.values())[0][:100]}...")

Loaded 500 questions and answers from MATH-500 dataset
Example question: Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the...
Example answer: \left( 3, \frac{\pi}{2} \right)...


In [4]:
# Load the model from safetensors using transformers (much lighter than vLLM)
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from pathlib import Path

# Model path - safetensors files are in the repo root
model_path = Path("/home/adisi/repos/py/inter_2025")

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    str(model_path),
    trust_remote_code=True
)

print("Loading model from safetensors...")
model = AutoModelForCausalLM.from_pretrained(
    str(model_path),
    torch_dtype=torch.float16,
    device_map="auto",  # Automatically handles GPU/CPU placement
    trust_remote_code=True
)

print("Model loaded successfully!")


/home/adisi/.conda/envs/inter_251208/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading model from safetensors...


Loading checkpoint shards: 100%|██████████| 2/2 [01:32<00:00, 46.38s/it]

Model loaded successfully!


In [ ]:
# Run the model on all MATH-500 queries and save the generated text (including thinking process)
from pathlib import Path
import json

# Load the prompt template
prompt_template_path = Path("/home/adisi/repos/py/inter_2025/prompts/play/question_formats/chain_of_thought.md")
with open(prompt_template_path, 'r', encoding='utf-8') as f:
    prompt_template = f.read()

# Create output directory
output_dir = Path("/home/adisi/repos/py/inter_2025/prompts/play/injections/irrelevant")
output_dir.mkdir(parents=True, exist_ok=True)

# Generation parameters
generation_config = {
    "temperature": 0.7,
    "top_p": 0.9,
    "max_new_tokens": 2048,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

print(f"Running model on {len(math_500_dict)} queries...")
print(f"Output directory: {output_dir}")

# Process each query
for idx, (question, correct_answer) in enumerate(math_500_dict.items()):
    print(f"Processing query {idx + 1}/{len(math_500_dict)}")
    # Format the prompt
    prompt = prompt_template.format(question=question)
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_config
        )
    
    # Decode the generated text (including the input prompt)
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Extract only the generated portion (after the prompt)
    prompt_tokens = tokenizer.encode(prompt, return_tensors="pt")
    generated_tokens = outputs[0][len(prompt_tokens[0]):]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=False)
    
    # Save to file
    output_file = output_dir / f"thought_{idx:04d}.txt"
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(generated_text)
    
    # Progress update
    if (idx + 1) % 50 == 0:
        print(f"  Processed {idx + 1}/{len(math_500_dict)} queries...")

print(f"\nDone! Saved {len(math_500_dict)} generated texts to {output_dir}")
